# Resolución ejercicio 3.10.29
## Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1

Show the step-by-step execution of Stages and of UniStages in the ring of Figure 3.3. Indicate for each step, the values know at the candidates.

Se asumen las restricciones R + ID:
- Links bidireccionales
- Conectividad
- Total reliability
- Valores iniciales distintos

Si bien Conectividad no es necesaria pues la topologia la restringimos a un anillo de n nodos como indica la figura 3.3:
- Anillo

In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger
from pydistsim.network.behavior import NetworkBehaviorModel
from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt
import time


## Stages

### Implementacion

In [ ]:
class Stages(NodeAlgorithm):
    default_params = {
        "election" : "Election",
        "notify" : "Notify",
    }

    class Status(StatusValues):
        ASLEEP = "ASLEEP",
        CANDIDATE = "CANDIDATE",
        WAITING = "WAITING",
        DEFEATED = "DEFEATED",
        FOLLOWER = "FOLLOWER",
        LEADER = "LEADER"

    S_init = (Status.ASLEEP)
    S_term = (Status.FOLLOWER, Status.LEADER)


    algorithm_restrictions = (
        BidirectionalLinks, Connectivity, TotalReliability, InitialDistinctValues
    )

    def initialize(self, node: NodeAccess):
        node.memory["stage"] = 1
        node.memory["count"] = 0
        node.memory["min"] = node.memory["unique_value"]
        self.send(
            node,
            data= [node.memory["unique_value"], node.memory["stage"]],
            destination= list(node.neighbors()),
            header= self.default_params["election"],
        )
        pass

    def close(self, node: NodeAccess, neighbor):
        node.memory["closed"].add(neighbor)
        node.memory[neighbor] = []

    def open(self, node: NodeAccess, neighbor):
        if neighbor in node.memory["closed"]:
            node.memory["closed"].remove(neighbor)
            for m in node.memory[neighbor]:
                node.push_to_inbox(m)
            node.memory[neighbor] = []
        
    def check_closed(self, node: NodeAccess, message : Message):
        if message.source in node.memory["closed"]:
            node.memory[message.source].append(message)
            return True 
        return False

    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , content: " + str(message.data)
        raise Exception(msj)


    def initializer(self):
        self.apply_restrictions()
        for node in self.network.nodes():
            node.status = self.Status.ASLEEP
            node.memory["min"] = None
            node.memory["stage"] = 0
            node.memory["count"] = 0
            node.memory["closed"] = set()
            node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))


    @Status.ASLEEP
    def spontaneously(self, node: NodeAccess, message: Message):
        self.initialize(node)
        node.status = self.Status.CANDIDATE


    @Status.ASLEEP
    def receiving(self, node: NodeAccess, message: Message):
        self.initialize(node)
        if message.header == self.default_params["election"]:
            id_ = message.data[0]
            node.memory["min"] = min(node.memory["min"], id_)
            self.close(node, message.source)
            node.status = self.Status.WAITING
        else:
            self.error("asleep - receiving", message)


    @Status.CANDIDATE
    def receiving(self, node: NodeAccess, message: Message):
        if(self.check_closed(node, message)):
            return
        if message.header == self.default_params["election"]:
            id_ = message.data[0]
            if id_ != node.memory["unique_value"]:
                node.memory["min"] = min(node.memory["min"], id_)
                self.close(node, message.source)
                node.status = self.Status.WAITING
            else:
                self.send(
                    node,
                    data= None,
                    destination= list(node.neighbors()),
                    header= self.default_params["notify"],
                )
                node.status = self.Status.LEADER
        else:
            self.error("candidate - receiving", message)

    @Status.WAITING
    def receiving(self, node: NodeAccess, message: Message):
        if(self.check_closed(node, message)):
            return        
        if message.header == self.default_params["election"]:
            other = [o for o in node.neighbors() if o != message.source][0]
            self.open(node, other)
            node.memory["stage"] += 1
            id_ = message.data[0]
            node.memory["min"] = min(node.memory["min"], id_)
            if node.memory["min"] == node.memory["unique_value"]:
                self.send(
                    node,
                    data= [node.memory["unique_value"], node.memory["stage"]],
                    destination= list(node.neighbors()),
                    header= self.default_params["election"],
                )
                node.status = self.Status.CANDIDATE
            else:
                node.status = self.Status.DEFEATED
        else:
            self.error("waiting - receiving", message)
    

    @Status.DEFEATED
    def receiving(self, node: NodeAccess, message: Message):
        if(self.check_closed(node, message)):
            return
        self.send(
            node,
            data= message.data,
            destination=[other for other in node.neighbors() if other != message.source],
            header= message.header,
        )
        if message.header == self.default_params["notify"]:
            node.status = self.Status.FOLLOWER

    @Status.LEADER
    def default(self, *args, **kwargs):    
        pass
    
    @Status.FOLLOWER
    def default(self, *args, **kwargs):    
        pass

### Ejecución


In [ ]:
net = NetworkGenerator().generate_ring_network(n = 14)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (Stages,)
set_log_level(LogLevels.INFO)
enable_logger()


In [ ]:
sim.run()
# fig = draw.draw_current_state(sim)
# fig

### Pruebas

In [ ]:
ordered_nodes = sorted(net.nodes(), key=lambda n: n.memory["unique_value"])
assert ordered_nodes[0].status == Stages.Status.LEADER
assert all(n.status == Stages.Status.FOLLOWER for n in ordered_nodes[1:])
print("OK")

## UniStages


### Implementacion

In [ ]:
class UniStages(NodeAlgorithm):
    default_params = {
        "election" : "Election",
        "notify" : "Notify",
    }

    class Status(StatusValues):
        ASLEEP = "ASLEEP",
        CANDIDATE = "CANDIDATE",
        DEFEATED = "DEFEATED",
        FOLLOWER = "FOLLOWER",
        LEADER = "LEADER"

    S_init = (Status.ASLEEP)
    S_term = (Status.FOLLOWER, Status.LEADER)


    algorithm_restrictions = (
        Connectivity, TotalReliability, InitialDistinctValues
    )

    def initialize(self, node: NodeAccess):
        node.memory["stage"] = 1
        node.memory["count"] = 0
        node.memory["order"] = 0
        node.memory["value1"] = node.memory["unique_value"]
        self.send(
            node,
            data= [node.memory["value1"], node.memory["stage"], node.memory["order"]],
            destination= list(node.neighbors()),
            header= self.default_params["election"],
        )

    def process_message(self, node: NodeAccess, value_, stage_, order_):
        if stage_ == node.memory["stage"]:
            if order_ == 0:
                node.memory["envelope"] = value_
                node.memory["order"] = 1
                self.send(
                    node,
                    data= [value_, stage_, node.memory["order"]],
                    destination= list(node.neighbors()),
                    header= self.default_params["election"],
                )
            else:
                node.memory["value2"] = value_
            node.memory["count"] += 1
            if node.memory["count"] == 2:
                if node.memory["envelope"] < min(node.memory["value1"], node.memory["value2"]):
                    node.memory["order"] = 0
                    node.memory["count"] = 0
                    node.memory["stage"] += 1
                    node.memory["value1"] = node.memory["envelope"];
                    self.send(
                        node,
                        data= [node.memory["value1"], node.memory["stage"], node.memory["order"]],
                        destination= list(node.neighbors()),
                        header= self.default_params["election"],
                    )
                else:
                    node.status = self.Status.DEFEATED
        elif stage_ > node.memory["stage"]:
            self.send(
                node,
                data= [value_, stage_, order_],
                destination= list(node.neighbors()),
                header= self.default_params["election"],
            )
            node.status = self.Status.DEFEATED;

    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , type: " + str(message.header)
        raise Exception(msj)


    def initializer(self):
        self.apply_restrictions()
        for node in self.network.nodes():
            node.status = self.Status.ASLEEP
            node.memory["stage"] = None
            node.memory["count"] = None
            node.memory["order"] = None
            node.memory["value1"] = None
            node.memory["value2"] = None
            node.memory["envelope"] = None
            node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))


    @Status.ASLEEP
    def spontaneously(self, node: NodeAccess, message: Message):
        self.initialize(node)
        node.status = self.Status.CANDIDATE

    @Status.ASLEEP
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            self.send(
                node,
                data= message.data,
                destination= list(node.neighbors()),
                header= self.default_params["election"],
            )
            node.status = self.Status.DEFEATED
        else:
            self.error("asleep - receiving", message)


    @Status.CANDIDATE
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["election"]:
            value_ = message.data[0]
            if value_ != node.memory["value1"]:
                stage_ = message.data[1]
                order_ = message.data[2]
                self.process_message(node, value_, stage_, order_)
            else:
                self.send(
                    node,
                    data= None,
                    destination= list(node.neighbors()),
                    header= self.default_params["notify"],
                )
                node.status = self.Status.LEADER
        else:
            self.error("candidate - receiving", message)

    @Status.DEFEATED
    def receiving(self, node: NodeAccess, message: Message):
        self.send(
            node,
            data= message.data,
            destination= list(node.neighbors()),
            header= message.header,
        )
        if message.header == self.default_params["notify"]:
            node.status = self.Status.FOLLOWER
    
    @Status.LEADER
    def default(self, *args, **kwargs):    
        pass

    @Status.FOLLOWER
    def default(self, *args, **kwargs):    
        pass

### Ejecucion

In [ ]:
net = NetworkGenerator(directed=True).generate_ring_network(n = 14, directed_network = True)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (UniStages,)
set_log_level(LogLevels.INFO)
enable_logger()

In [ ]:
sim.run(1)
fig = draw.draw_current_state(sim)
fig


In [ ]:
# ordered_nodes = sorted(net.nodes(), key=lambda n: n.memory["unique_value"])
# assert ordered_nodes[0].status == Stages.Status.LEADER
# assert all(n.status == Stages.Status.FOLLOWER for n in ordered_nodes[1:])
# print("OK")
for node in net.nodes():
    print(node.memory['unique_value'])
    # print(f"Node {node} with value {node.memory['unique_value']} ended as {node.status}")

## Otros

In [ ]:
sim.reset()
plt.close()